import pandas as pd
from pathlib import Path
from src.data_import import load_data
from src.data_cleaning import select_columns, strip_spaces_columns, normalize_case_columns, gene_filter, convert_numeric_columns, drop_missing_required, check_influence_cooks_distance
from src.statistical_analysis import factor_categorical
from src.app_logger import logger

In [ ]:
import pandas as pd
from pathlib import Path
from src.data_import import load_data
from src.data_cleaning import select_columns, strip_spaces_columns, normalize_case_columns, gene_filter, convert_numeric_columns, drop_missing_required, check_influence_cooks_distance
from src.statistical_analysis import factor_categorical
from src.app_logger import logger

ImportError: cannot import name 'check_influence_cooks_distance' from 'src.data_cleaning' (/Users/carolfarah/final_projext_yas/final_project/src/data_cleaning.py)

In [ ]:
columns_list = ["Patient_ID", "Gene/Factor", "Disease_Stage", "Brain_Volume_Loss", "Age", "Sex"]

sub_df = select_columns(df, columns_list) #Columns from original df were selected
sub_df = strip_spaces_columns(sub_df, columns=["Gene/Factor", "Disease_Stage", "Sex"])
sub_df = normalize_case_columns(sub_df, columns= ["Gene/Factor", "Disease_Stage", "Sex"])
sub_df = gene_filter(sub_df, "Gene/Factor", values_list= ["mlh1", "msh3", "htt (somatic expansion)"])
sub_df = drop_missing_required(sub_df, columns_list)
clean_df, removed_rows, threshold= check_influence_cooks_distance(sub_df, "Brain_Volume_Loss", "Age", "Sex")
sub_df = factor_categorical(sub_df, "Disease_Stage", "Sex")

In [ ]:
from src.statistical_assumptions import check_independence_duplicates, plot_ancova_linearity, drop_duplicate_subjects, levene_two_way_anova, levene_ancova, check_linearity_age_dv, log_transform, check_homogeneity_of_slopes
from src.data_visualization import plot_ancova_linearity
independence_test = check_independence_duplicates(sub_df, "Patient_ID")
if independence_test.empty:
    print("No duplicates, observations are all independent")
else:
    print(f"Duplicates detected! There are {len(independence_test)/2}")
    sub_df = drop_duplicate_subjects(sub_df, "Patient_ID", keep= "first")
    print(f"{len(independence_test)/2} rows removed!")


In [ ]:
linearity_check = check_linearity_age_dv(df, dv="Brain_Volume_Loss", cov="Age", show_plot=True)
linearity_check_p_value = linearity_check["p_value"]


In [ ]:
ancova_levene_stat, ancova_levene_p = levene_ancova(sub_df, "Brain_Volume_Loss", "Disease_Stage", "Age", center='median')


In [ ]:
ancova_linearity_graphs = plot_ancova_linearity(sub_df, dv="Brain_Volume_Loss", iv="Disease_Stage", cov="Age")

while True:
    transform_dataset = input("Does dataset require transformation? (yes/no): ").strip().lower()
    
    if transform_dataset in {"yes", "no"}:
        break
    else:
        print("Please enter 'yes' or 'no'.")

if transform_dataset == "yes":
    clean_df = log_transform(clean_df, "Brain_Volume_Loss", new_column = "Brain_Volume_Loss") #replaces values with new values after transformation
    clean_df = log_transform(clean_df, "Age", new_column = "Age")
    ancova_linearity_graphs = plot_ancova_linearity(sub_df, dv="Brain_Volume_Loss", iv="Disease_Stage", cov="Age")
else:
    print("Proceeding with assumptions analysis")


In [ ]:
homogeneity_of_slopes_table = check_homogeneity_of_slopes(sub_df, "Brain_Volume_Loss", "Disease_Stage", "Age")
p_val_homogeneity_of_slopes= anova_table.loc["C(Disease_Stage):Age", "PR(>F)"]

p_iv  = ancova_table.loc["C(Disease_Stage)", "PR(>F)"]
p_cov = ancova_table.loc["Age", "PR(>F)"]

iv_sig  = p_iv  < 0.05
cov_sig = p_cov < 0.05

from src.statistical_analysis import run_ancova, run_ancova_with_statsmodels_posthoc, run_moderated_regression

if p_val_homogeneity_of_slopes > 0.05:
    print("The effect of the covariate Age are the same on the level of IV Disease_Stage.\n Conducting ANCOVA")
    ancova_test_model, ancova_test_table = run_ancova(sub_df, "Brain_Volume_Loss", "Disease_Stage", "Age", ancova_levene_p, linearity_check_p_value, alpha=0.05)
    if not iv_sig and not cov_sig:
        print("An ANCOVA revealed no significant effect of disease stage on brain volume loss, controlling for age," \
        "nor was age significantly associated with brain volume loss.\n")

    elif iv_sig and not cov_sig:
        print("An ANCOVA revealed a significant effect of disease stage on brain volume loss, " \
        f"controlling for age p = {p_iv}, ηp² = {ancova_table_table.iloc[0]["partial_eta_sq"]}. Age was not significantly associated with brain volume loss.")
        print("\nPost-hoc pairwise comparisons of adjusted means were conducted using Bonferroni correction.")
        run_posthoc = True

    elif not iv_sig and cov_sig:
        print("An ANCOVA revealed no significant effect of disease stage on brain volume loss after controlling for age. " \
        f"Age was significantly associated with brain volume loss, p = {ancova_table_table.iloc[1]["PR(>F)"]}.")

    else:
        print("An ANCOVA revealed a significant effect of disease stage on brain volume loss, " \
        f"controlling for age, F(df₁, df₂) = X.XX, p = {p_iv}, ηp² = {ancova_table_table.iloc[0]["partial_eta_sq"]}." \ 
        f"Age was also significantly associated with brain volume loss, F(1, df₂) = X.XX, p = {p_cov}.")
        print("\nPost-hoc pairwise comparisons of adjusted means were conducted using Bonferroni correction.")
        run_posthoc = True

    if run_posthoc = True:
        print("\nPost-hoc pairwise comparisons were conducted using Bonferroni correction.")
        ancova_post_hoc = run_ancova_with_statsmodels_posthoc(sub_df, "Brain_Volume_Loss", "Disease_Stage", "Age", alpha=0.05)
    
    print("The results of the post-hoc are as follows:\n" + ancova_post_hoc)
else: #Moderated regression
    print("The effect of the covariate Age differs depending on the level of IV Disease_Stage.\n Conducting moderated regression instead")
    moderated_regression_results = run_moderated_regression(sub_df, "Brain_Volume_Loss", "Disease_Stage", "Age")

    if "IV:Covariate" in moderated_regression_results.index and moderated_regression_results.loc["IV:Covariate", "P>|t|"] < 0.05:
    print("Interaction significant \nRunning spotlight/simple slopes at ±1 SD of Covariate")
    spotlight_analysis_results = run_spotlight_analysis(sub_df, "Brain_Volume_Loss", "Disease_Stage", "Age")

    print("Spotlight Analysis (Simple Slopes)\n")
    print(spotlight_analysis_results)

    elif "IV" in moderated_regression_results.index and moderated_regression_results.loc["IV", "P>|t|"] < 0.05:
    if df[iv].nunique() > 2:
        print("IV main effect significant\n running pairwise post-hoc comparisons between IV levels")
    else:
        print("IV main effect significant\n post-hoc needed (only 2 levels)")


